<a href="https://colab.research.google.com/github/DHKing7/Final_Project/blob/main/SCNN_Lane_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/example

/content/drive/MyDrive/example


In [3]:
ls

SDLane/


# JSON Key 확인

In [3]:
import json
import os

In [ ]:
cd SDLane/train/labels

/content/drive/MyDrive/SDLane/train/labels


In [ ]:
ls

5bbb9c8402a48a0f7eab3af7692743953136faf4/


In [ ]:
cd 5bbb9c8402a48a0f7eab3af7692743953136faf4/

/content/drive/MyDrive/SDLane/train/labels/5bbb9c8402a48a0f7eab3af7692743953136faf4


In [ ]:
ls

0001.json  0246.json  0491.json  0736.json  0981.json  1226.json  1497.json
0002.json  0247.json  0492.json  0737.json  0982.json  1227.json  1498.json
0003.json  0248.json  0493.json  0738.json  0983.json  1228.json  1499.json
0004.json  0249.json  0494.json  0739.json  0984.json  1229.json  1500.json
0005.json  0250.json  0495.json  0740.json  0985.json  1230.json  1501.json
0006.json  0251.json  0496.json  0741.json  0986.json  1231.json  1502.json
0007.json  0252.json  0497.json  0742.json  0987.json  1232.json  1503.json
0008.json  0253.json  0498.json  0743.json  0988.json  1233.json  1504.json
0009.json  0254.json  0499.json  0744.json  0989.json  1234.json  1505.json
0010.json  0255.json  0500.json  0745.json  0990.json  1235.json  1506.json
0011.json  0256.json  0501.json  0746.json  0991.json  1236.json  1507.json
0012.json  0257.json  0502.json  0747.json  0992.json  1237.json  1508.json
0013.json  0258.json  0503.json  0748.json  0993.json  1238.json  1509.json
0014.json  0

In [ ]:
json_path = "0001.json"

In [ ]:
with open(json_path, "r") as f: # json_path 수정
  annotation = json.load(f)

In [ ]:
len(annotation["geometry"])

4

In [ ]:
annotation["idx"]

[1, 2, 3, 4]

In [4]:
import torch
from torch.utils.data import Dataset
import numpy as np
import cv2

In [15]:
class SDLane(Dataset):
  def __init__(self, path, image_set, transforms = None):
    super(SDLane, self).__init__()
    assert image_set in ('train', 'test'), "image_set is not valid!" # 'val'은 없음
    self.data_dir_path = path # SDLane/
    self.image_set = image_set # train/ or test/
    self.tramsforms = transforms
    

    if not os.path.exists(os.path.join(path, image_set, "seg_labels")): # SDLane/train/seg_label
      print("Label is going to get generated into dir: {} ...".format(os.path.join(path, "seg_labels")))
      self.generate_label()
    self.createIndex()



  def createIndex(self):
    self.img_list = [] # img 파일명
    self.segLabel_list = [] # ["Geometry"] 정보 그그그 json 열고 세그멘테이션 이미지 변환한거
    self.exist_list = [] # 차선 존재 유무 42dot dataset은 5개 이상 일 수 있음

    listfile = os.path.join(self.data_dir_path, "train_list.txt") # txt파일에 images/0932b1d66d21e2ce4de81086645ebd93955fb0c1/0001.jpg 형식으로 39000개 존재
    if not os.path.exists(listfile):
      raise FileNotFoundError("List file doesn't exist. Label has to be generated!")
    
    with open(listfile) as f: ### 리스트 파일 열어서 한줄 씩 lable,img 리스트에 추가
      for line in f:
        line = line.strip()
        l = line.split("/") #["images", "0932b1d66d21e2ce4de81086645ebd93955fb0c1", "0001.jpg"]

        json_path = os.path.join(self.data_dir_path, "train/labels", l[1], l[2][:-4]+".json") # SDLane/train/seg_labels/0932b1d66d21e2ce4de81086645ebd93955fb0c1/0001.json
        with open(json_path, "r") as f: # json_path 수정
          annotation = json.load(f)


        self.img_list.append(os.path.join(self.data_dir_path, self.image_set, line)) # SDLane/train/images/0932b1d66d21e2ce4de81086645ebd93955fb0c1/0001.jpg
        self.segLabel_list.append(os.path.join(self.data_dir_path, self.image_set,"seg_labels", l[1], l[2])) # SDLane/train/seg_labels/0932b1d66d21e2ce4de81086645ebd93955fb0c1/0001.jpg
        self.exist_list.append(annotation["idx"]) ### json 파일을 까야함. 차선 존재 유무 SDLane은 차선 수가 변동적임
  
  def __getitem__(self, idx):
    img = cv2.imread(self.img_list[idx])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    if self.image_set != "test":
      segLabel = cv2.imread(self.segLabel_list[idx])[:,:,0] # ??
      exist = np.array(self.exist_list[idx])
    else:
      segLabel = None
      exist = None


    sample = {"img": img,
              "segLable": segLabel,
              "exist": exist,
              "img_name": self.img_list[idx]}
    
    if self.transform is not None:
      sample = self.transforms(sample)

    return sample

  def __len__(self):
    return len(self.img_list)

  def generate_label(self):
    save_dir = os.path.join(self.data_dir_path, self.image_set, "seg_labels") #SDLane/train/seg_labels/
    os.makedirs(save_dir, exist_ok = True)

    self._gen_label_for_json('train', vis = None)


  def _gen_label_for_json(self, image_set, vis = None):
    H, W = 1208, 1920

    save_dir = "seg_labels"

    listf = os.path.join(self.data_dir_path, "train_list.txt")
    if not os.path.exists(listf):
      raise FileNotFoundError("List file doesn't exist. Label has to be generated!")

    with open(listf) as f: ### 리스트 파일 열어서 한줄 씩 lable,img 리스트에 추가
      for line in f:
        line = line.strip()
        l = line.split("/") #["images", "0932b1d66d21e2ce4de81086645ebd93955fb0c1", "0001.jpg"]

        json_path = os.path.join(self.data_dir_path, self.image_set,"labels", l[1], l[2][:-4]+".json") #SDLane/train/labels/0932b1d66d21e2ce4de81086645ebd93955fb0c1/0001.json
        
        seg = os.path.join(self.data_dir_path,"train/seg_labels",l[1]) #SDLane/train/seg_labels/0932b1d66d21e2ce4de81086645ebd93955fb0c1/
        os.makedirs(seg, exist_ok=True)


    ### json 파일 불러오기 key = "geometry", "idx"
        with open(json_path, "r") as f: # json_path 수정
          annotation = json.load(f)

        if vis is None:
          vis = np.zeros((H,W,3), dtype = np.uint8)
          vis = np.ascontiguousarray(vis)


        lane_geometry = annotation["geometry"]

        pts = [np.int32(lane) for lane in lane_geometry if not len(lane) == 0]
        vis = cv2.polylines(vis, pts, False, (0, 255, 0), 10) # vis -> seg_label로 이동시켜야함
        seg_path = os.path.join(seg, l[2]) #SDLane/train/seg_labels/0932b1d66d21e2ce4de81086645ebd93955fb0c1/0001.jpg
        cv2.imwrite(seg_path, vis) #
        vis = None 


In [16]:
dataset = SDLane("SDLane", "train", transforms = None)

Label is going to get generated into dir: SDLane/seg_labels ...


AttributeError: ignored

In [12]:
dataset.__len__()

20

In [13]:
dataset.__getitem__(1)

AttributeError: ignored